<p style="font-size:14px; text-align: right">CoastWatch Python Exercises</p>  

# Virtual Station Tutorial
> history | updated August 2021  
> owner | NOAA CoastWatch

In this exercise, you will import a ship track and extract satellite SST values at locations along the track.
The exercise demonstrates the following skills:

*    Getting data from a remote CSV file
*    Downloading satellite data from ERDDAP in netCDF format
*    Extracting data with Python


# Get set up
## Import some basic python modules

In [1]:
import pkg_resources


## Look for python modules you might not have installed
We will be using the xarray, numpy, and pandas modules for this exercise. Make sure that they are installed in your Python 3 environment. A quick way to do this is with the script below

In [2]:
# Create a set 'curly brackets' of the modules to look for
# You can put any modules that you want to in the set
required = {'xarray', 'numpy', 'pandas'}

# look for the installed packages
installed = {pkg.key for pkg in pkg_resources.working_set}

# Find which modules are missing
missing = required - installed
if len(missing)==0:
    print('All modules are installed')
else:
    print('These modules are missing', ', '.join(missing))

All modules are installed


#### If you find missing modules, please go to the appendix at the bottom of the page for installation instructions.

## Import the modules

In [3]:
import numpy as np  # for matrix operations
import numpy.ma as ma  # for masking arrays
import pandas as pd  # for tabular data
import xarray as xr  # for gridded data

# Generate means for a ship station track 

## Load a station location file (.csv) with the time, latitude, and longitude coordinates for station locations

For this example the CSV file has three columns, with each row locating a station in time and space:  
`sample_date	sample_time	sample_lat	sample_lon`  
`6/3/20	        18:00	    37.3	    -123.2`

### Load the CSV file data using the "read_csv" function from the Pandas module.  
To use the date and time, we will need to create a Pandas date object. Pandas has a nice option (parse_dates) to create a column date object from the date and time fields as the CSV file is loaded. We will also keep the original sample_date and sample_time by setting the keep_date_col to True.

### The Xarray module requires dates be formatted in a ISO 8601 date format, so we will also create an iso_date column

In [9]:
# Load the file into a Pandas data frame
# This creates a date object column named 'sample_date_sample_time' 
track_DF = pd.read_csv('data/my_monterey_track.csv', parse_dates=[['sample_date', 'sample_time']], keep_date_col=True) 


# track_DF['iso_date'] = track_DF.sample_date_sample_time.dt.strftime('%Y-%m-%dT12:%M:%S')
track_DF['iso_date'] = track_DF.sample_date_sample_time.dt.strftime('%Y-%m-%d')
# Display the data
display(track_DF)

,sample_date_sample_time,sample_date,sample_time,sample_lat,sample_lon,iso_date
0,2020-06-03 18:00:00,6/3/20,18:00,37.3,236.8,2020-06-03
1,2020-06-04 19:34:00,6/4/20,19:34,36.1,237.5,2020-06-04
2,2020-06-05 20:15:00,6/5/20,20:15,34.4,238.2,2020-06-05
3,2020-06-06 18:45:00,6/6/20,18:45,32.2,239.8,2020-06-06
4,2020-06-07 12:52:00,6/7/20,12:52,31.9,240.9,2020-06-07
5,2020-06-08 14:23:00,6/8/20,14:23,31.9,241.8,2020-06-08


## Load the SST data from ERDDAP (detailed explanations in tutorial #1)

In [4]:
# Define the function
def point_to_dataset(dataset_id, base_url='https://coastwatch.pfeg.noaa.gov/erddap/griddap'):
    base_url = base_url.rstrip('/')
    full_url = '/'.join([base_url, dataset_id])
    #print(full_url)
    return xr.open_dataset(full_url)

# Call the function
da = point_to_dataset('nesdisGeoPolarSSTN5SQNRT')

# uncomment the "da" below to see the full data array info
# da

In [5]:
def get_data(my_da, my_var,
             my_lt_min, my_lt_max, 
             my_ln_min, my_ln_max, 
             my_tm_min, my_tm_max
            ):
    
    my_data = my_da[my_var].sel(
                                latitude=slice(my_lt_min, my_lt_max), 
                                longitude=slice(my_ln_min, my_ln_max), 
                                time=slice(my_tm_min, my_tm_max)
                               )
    return my_data

### Run the function with our geographical and time ranges

In [6]:
lat_min  = 32.
lat_max = 39.
lon_min = -124.
lon_max = -117.
time_min = '2020-06-03T12:00:00'  # written in ISO format
time_max = '2020-06-07T12:00:00'  # written in ISO format
my_var = 'analysed_sst'

sst = get_data(
               da, my_var,
               lat_min, lat_max,
               lon_min, lon_max,
               time_min, time_max
              )

# the sst data array is a subset of da
print(sst.dims)
print('dimension size', sst.shape)

# uncomment the "sst" below to see the full data array info
#sst

('time', 'latitude', 'longitude')
dimension size (5, 140, 140)


## Make the longitude format compatible with your SST dataset
### Some datasets use a -180°E to +180°E format  
* The 0° longitude is at the prime meridian (the location of the Royal Observatory, Greenwich, in London, England).  
* Longitudes west of 0° have a negative value.  
* Longitudes east of 0° have a positive value

### Some datasets use a 0°E to 360°E format  
* The 0° longitude is at the prime meridian  
* All longitudes have a positive value, increasing in value eastward of 0°.  

### The datasets used in this example have different longitude formats 
* Our station location longitudes above have values > 180, so they are using a 0 to 360 longitude format. 
* The satellite SST data product we will be using has longitudes in a -180 to +180 format.  
longitude range: -179.975 to 179.975

Your station location longitudes need to be in the same format as the SST data longitudes. We can write a quick script to check if the dataset or the station location file have negative numbers, indicating a -180 to +180 longitude.  
* If both longitude lists have the same format, then the station location longitudes are returned unchanged.
* If the SST longitudes are -180 to 180 and the station location longitudes are 0-360, then 360 is subtracted from any station location longitude > 180
* If the SST longitudes are 0 to 360 and the station location longitudes are -180 to 180, then 360 is added to any station location longitude < 0  

### Create a function to correct the longitude format
Where:  
dataset_lon is the longitude column from the xarray data array ( sst['longitude'] )  
location_lon is the longitude column from the Pandas data frame, ( track_DF['sample_lon'] )  

In [7]:
def is_it_180_to_180(dataset_lon, location_lon):
    #long_array = False
    #long_array = np.any((da.longitude > 0))
    datasat_is = np.any((dataset_lon < 0))
    location_is = np.any((location_lon < 0))
    if datasat_is and location_is:
        return location_lon
    
    elif not datasat_is and location_is:
        return [ x if x>0 else x+360 for x in location_lon ]
    elif datasat_is and not location_is:
        return [ x if x<=180 else x-360 for x in location_lon ]


### Apply the longitude format correction function
Pass the dataset longitudes and the station location longitudes to the function and add the results to the Pandas data frame with the station information, with this usage:

In [10]:
track_DF['sample_lon_corrected'] = is_it_180_to_180(
                                                    sst['longitude'], 
                                                    track_DF['sample_lon']
                                                   )

track_DF

,sample_date_sample_time,sample_date,sample_time,sample_lat,sample_lon,iso_date,sample_lon_corrected
0,2020-06-03 18:00:00,6/3/20,18:00,37.3,236.8,2020-06-03,-123.2
1,2020-06-04 19:34:00,6/4/20,19:34,36.1,237.5,2020-06-04,-122.5
2,2020-06-05 20:15:00,6/5/20,20:15,34.4,238.2,2020-06-05,-121.8
3,2020-06-06 18:45:00,6/6/20,18:45,32.2,239.8,2020-06-06,-120.2
4,2020-06-07 12:52:00,6/7/20,12:52,31.9,240.9,2020-06-07,-119.1
5,2020-06-08 14:23:00,6/8/20,14:23,31.9,241.8,2020-06-08,-118.2


### Select an area around each ship station to get satellite data
* If you just use a single latitude and longitude pair to extract SST data, then the data from a single point will be returned. 
* To increase to chances of getting data (if there are gaps due to clouds for example) and improve your statistics, you can request a box of SST data around the station location, e.g. a latitude range and a longitude range.  

The box can increase by multiples of the spatial resolution of the dataset. For this dataset the spatial resolution was 0.05 degrees for the latitude and longitude. That is approximately 5 km resolution. So, a 5 pixel X 5 pixel box around each station location would result in a 25 x 25 km box (625 km^2) of data.  

Let's build a function to generate a latitude and longitude range box for each station location. The function asks for the following:  
* the station latitude and longitude
* The latitude and longitude spatial resolution of the satellite dataset. We obtained these values from the satellite dataset metadata.
    * da.geospatial_lat_resolution
    * da.geospatial_lon_resolution
* how many pixels you want added to each side of the latitude and longitude center point  
    * For example if you choose "2" for the latitude, then you will get a latitude range that covers five pixels: the station latitude for the center plus two pixel on each side. If you also choose "2" for the longitude, then you would get a 5x5 box around the station location. 
    * The default value for the function is "1", which is a 3x3 box  

In [11]:
def data_box(
             st_lat, st_lon, 
             lat_res, lon_res, 
             lat_pixels = 1, lon_pixels = 1
            ):
    
    lt_min = st_lat - lat_res * lat_pixels
    lt_max = st_lat + lat_res * lat_pixels
    ln_min = st_lon - lon_res * lon_pixels
    ln_max = st_lon + lon_res * lon_pixels

    return lt_min, lt_max, ln_min, ln_max
    

### Pass the dataset longitudes and the station location longitudes to the function  

Add the results to the Pandas data frame with the station information, with this usage:

new_Pandas_dataframe_column = data_box(st_lat, st_lon, lat_res, lon_res, lat_pixels = 1, lon_pixels = 1)

where: 
* new_Pandas_dataframe_columns = track_DF['lat_min'], track_DF['lat_min'], track_DF['lon_min'], track_DF['lon_min']
* st_lat = the latitude column from the Pandas station location data frame, track_DF['sample_lat']
* st_lon = the longitude column from the Pandas station location data frame, track_DF['sample_lon_corrected']
    * make sure to use the corrected longitude column
* lat_res = the spatial resolution for satellite latitude, da.geospatial_lat_resolution
* lon_res = the spatial resolution for satellite longitude, da.geospatial_lon_resolution 
* lat_pixels = the number of pixels to add to each side of the latitude center point (default = 1) 
* lon_pixels = the number of pixels to add to each side of the longitude center point (default = 1)

In [12]:
# use the data_box function
[
 track_DF['lat_min'], track_DF['lat_max'],
 track_DF['lon_min'], track_DF['lon_max']
] = data_box(
             track_DF['sample_lat'], track_DF['sample_lon_corrected'],
             da.geospatial_lat_resolution, da.geospatial_lon_resolution,
             2, 2
            )
                                                                                              
track_DF                                                                                            

,sample_date_sample_time,sample_date,sample_time,sample_lat,sample_lon,iso_date,sample_lon_corrected,lat_min,lat_max,lon_min,lon_max
0,2020-06-03 18:00:00,6/3/20,18:00,37.3,236.8,2020-06-03,-123.2,37.2,37.4,-123.3,-123.1
1,2020-06-04 19:34:00,6/4/20,19:34,36.1,237.5,2020-06-04,-122.5,36.0,36.2,-122.6,-122.4
2,2020-06-05 20:15:00,6/5/20,20:15,34.4,238.2,2020-06-05,-121.8,34.3,34.5,-121.9,-121.7
3,2020-06-06 18:45:00,6/6/20,18:45,32.2,239.8,2020-06-06,-120.2,32.1,32.3,-120.3,-120.1
4,2020-06-07 12:52:00,6/7/20,12:52,31.9,240.9,2020-06-07,-119.1,31.8,32.0,-119.2,-119.0
5,2020-06-08 14:23:00,6/8/20,14:23,31.9,241.8,2020-06-08,-118.2,31.8,32.0,-118.3,-118.1


### Extract SST data for each station location and generate some basic statistics
#### We now have functions and the input parameters needed to obtain satellite data for the cruise track
* Build a "for" loop to use the sample_date_sample_time, lat_min, lat_max, lon_min, and lon_max columns from the track_DF in the get_data() function to download satellite data for each station.
* Calculate the mean, standard deviation, and n stats, then add them to the track_DF

In [13]:
means = []
stdevs = []
ns = []

for index, row in track_DF.iterrows():
    
    my_data = get_data(da, my_var,
                       row['lat_min'], row['lat_max'], 
                       row['lon_min'], row['lon_max'], 
                       row['iso_date'], row['iso_date']
                      )
    
    means.append(np.nanmean(my_data, axis=(1,2))[0])
    stdevs.append(np.nanstd(my_data, axis=(1,2))[0])
    ns.append(np.count_nonzero(my_data, axis=(1,2))[0])

print('means', means)
print('stdevs', stdevs)
print('ns', ns)

    

means [14.251244, 14.836244, 16.224995, 16.861244, 17.026245, 17.41437]
stdevs [0.3615224, 0.12873788, 0.09499985, 0.048589565, 0.09873407, 0.10271109]
ns [16, 16, 16, 16, 16, 16]


### Add the statistics to the track_DF dataframe
Uncomment the last line to save the dataframe as a CSV file called 'mbnms_results.csv'

In [14]:
track_DF['mean'] = means
track_DF['stdev'] = stdevs
track_DF['n'] = ns
display(track_DF)

# uncomment to save the data
# track_DF.to_csv('crusetrack_results.csv', index = False)

,sample_date_sample_time,sample_date,sample_time,sample_lat,sample_lon,iso_date,sample_lon_corrected,lat_min,lat_max,lon_min,lon_max,mean,stdev,n
0,2020-06-03 18:00:00,6/3/20,18:00,37.3,236.8,2020-06-03,-123.2,37.2,37.4,-123.3,-123.1,14.251244,0.361522,16
1,2020-06-04 19:34:00,6/4/20,19:34,36.1,237.5,2020-06-04,-122.5,36.0,36.2,-122.6,-122.4,14.836244,0.128738,16
2,2020-06-05 20:15:00,6/5/20,20:15,34.4,238.2,2020-06-05,-121.8,34.3,34.5,-121.9,-121.7,16.224995,0.095000,16
3,2020-06-06 18:45:00,6/6/20,18:45,32.2,239.8,2020-06-06,-120.2,32.1,32.3,-120.3,-120.1,16.861244,0.048590,16
4,2020-06-07 12:52:00,6/7/20,12:52,31.9,240.9,2020-06-07,-119.1,31.8,32.0,-119.2,-119.0,17.026245,0.098734,16
5,2020-06-08 14:23:00,6/8/20,14:23,31.9,241.8,2020-06-08,-118.2,31.8,32.0,-118.3,-118.1,17.414370,0.102711,16


# Appendix

## Installation instruction are at these links
pandas: https://pandas.pydata.org/pandas-docs/stable/getting_started/install.html   
numpy: https://numpy.org/install/  
xarray: http://xarray.pydata.org/en/latest/getting-started-guide/installing.html 